In [4]:
import cv2

def compare_images(img1_path, img2_path):
    # Load the two images
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    # Calculate the color histograms of the two images
    hist1 = cv2.calcHist([img1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([img2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

    # Normalize the histograms
    cv2.normalize(hist1, hist1)
    cv2.normalize(hist2, hist2)

    # Calculate the correlation between the two histograms
    correlation = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

    # Return the correlation value
    return correlation


In [5]:
import requests
from bs4 import BeautifulSoup
import os
import json

# Create a directory to store the images
if not os.path.exists('images'):
    os.mkdir('images')

# Iterate over the Pokemon in all pokedex files and download their images
for i in range(2, 23):
    pokedex_filename = f'pokedex/pokedex_{i:02d}.json'
    with open(pokedex_filename, 'r') as f:
        pokemon_list = json.load(f)

    for pokemon in pokemon_list:
        name = pokemon['name']
        url = f'https://pokemondb.net/pokedex/{name.lower()}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        image_url = soup.find('a', rel='lightbox')['href']
        pokemon['image_url'] = image_url

        # Download the image
        image_path = f'images/{name.lower()}.jpg'
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)



KeyboardInterrupt: 

In [3]:
import cv2

# Define a function to compare two Pokemon images and return their similarity score
def compare_pokemon_images(pokemon1_path, pokemon2_path):
    # Load the images
    pokemon1 = cv2.imread(pokemon1_path)
    pokemon2 = cv2.imread(pokemon2_path)
    
    # Resize the images to the same dimensions
    pokemon1_resized = cv2.resize(pokemon1, (300, 300))
    pokemon2_resized = cv2.resize(pokemon2, (300, 300))
    
    # Convert the images to grayscale
    pokemon1_gray = cv2.cvtColor(pokemon1_resized, cv2.COLOR_BGR2GRAY)
    pokemon2_gray = cv2.cvtColor(pokemon2_resized, cv2.COLOR_BGR2GRAY)
    
    # Calculate the difference between the images
    diff = cv2.absdiff(pokemon1_gray, pokemon2_gray)
    
    # Calculate the similarity score
    similarity_score = 1 - (diff.sum() / (300 * 300 * 255))
    
    return similarity_score

# Load the JSON file containing the Pokemon data
with open('pokedex/pokedex_01.json', 'r') as f:
    pokemon_list = json.load(f)

# Iterate over the Pokemon in the list and compare their images
for i in range(len(pokemon_list)):
    for j in range(i + 1, len(pokemon_list)):
        pokemon1_name = pokemon_list[i]['name'].lower()
        pokemon2_name = pokemon_list[j]['name'].lower()
        pokemon1_path = f'images/{pokemon1_name}.jpg'
        pokemon2_path = f'images/{pokemon2_name}.jpg'
        similarity_score = compare_pokemon_images(pokemon1_path, pokemon2_path)
        print(f'{pokemon1_name},{pokemon2_name},{similarity_score}')


NameError: name 'json' is not defined

In [1]:
import json
import os
import requests
from bs4 import BeautifulSoup
import cv2

# Load the JSON files containing the Pokemon data
pokemon_list = []
for i in range(1, 23):
    with open(f'pokedex/pokedex_{i:02d}.json', 'r') as f:
        pokemon_list += json.load(f)

# Create a directory to store the images
if not os.path.exists('images'):
    os.mkdir('images')

# Download the images and store them in the images directory
pokemon_list_with_images = []
for pokemon in pokemon_list:
    name = pokemon['name']
    url = f'https://pokemondb.net/pokedex/{name.lower()}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    image_url = soup.find('a', rel='lightbox')
    if image_url is not None:
        image_url = image_url['href']
        pokemon['image_url'] = image_url

        # Download the image
        image_path = f'images/{name.lower()}.jpg'
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Check if the image is valid
        if os.path.getsize(image_path) > 0:
            # Resize the image to reduce computation time
            image = cv2.imread(image_path)
            image = cv2.resize(image, (100, 100))

            # Add the Pokemon to the list
            pokemon['image_path'] = image_path
            pokemon_list_with_images.append(pokemon)

# Write the Pokemon list with images to a new JSON file
with open('pokedex_with_images.json', 'w') as f:
    json.dump(pokemon_list_with_images, f)


In [6]:
import json
import os
import cv2

# Load the JSON file containing the Pokemon data
with open('pokedex_with_images.json', 'r') as f:
    pokemon_list = json.load(f)

# Calculate the similarity between each pair of images
with open('similarity.txt', 'w') as f:
    for i in range(len(pokemon_list)):
        for j in range(i + 1, len(pokemon_list)):
            pokemon1 = pokemon_list[i]
            pokemon2 = pokemon_list[j]
            image1 = cv2.imread(pokemon1['image_path'])
            image2 = cv2.imread(pokemon2['image_path'])

            # Resize images to the same dimensions
            image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

            similarity = cv2.matchTemplate(image1, image2, cv2.TM_CCOEFF_NORMED)[0][0]
            f.write(f'{pokemon1["name"]},{pokemon2["name"]},{similarity}\n')


KeyboardInterrupt: 

In [7]:
# faster code for similarity check
import json
import os
import cv2
from multiprocessing import Pool

# Function to calculate similarity between two images
def calculate_similarity(pokemon1, pokemon2):
    image1 = cv2.imread(pokemon1['image_path'])
    image2 = cv2.imread(pokemon2['image_path'])

    # Resize images to the same dimensions
    image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

    similarity = cv2.matchTemplate(image1, image2, cv2.TM_CCOEFF_NORMED)[0][0]
    return pokemon1['name'], pokemon2['name'], similarity

if __name__ == '__main__':
    # Load the JSON file containing the Pokemon data
    with open('pokedex_with_images.json', 'r') as f:
        pokemon_list = json.load(f)

    # Calculate the similarity between each pair of images using multiprocessing
    with open('similarity.txt', 'w') as f, Pool() as pool:
        for pokemon1, pokemon2, similarity in pool.starmap(calculate_similarity, ((pokemon_list[i], pokemon_list[j]) for i in range(len(pokemon_list)) for j in range(i + 1, len(pokemon_list)))):
            f.write(f'{pokemon1},{pokemon2},{similarity}\n')


In [ ]:
# Calculate the similarity between each pair of images 
with open('gs_similarity.txt', 'w') as f:
    for i in range(len(pokemon_list)):
        for j in range(i + 1, len(pokemon_list)):
            pokemon1 = pokemon_list[i]
            pokemon2 = pokemon_list[j]
            image1 = cv2.imread(pokemon1['image_path'], cv2.IMREAD_GRAYSCALE)
            image2 = cv2.imread(pokemon2['image_path'], cv2.IMREAD_GRAYSCALE)

            # Resize images to the same dimensions
            image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

            similarity = cv2.matchTemplate(image1, image2, cv2.TM_CCOEFF_NORMED)[0][0]
            f.write(f'{pokemon1["name"]},{pokemon2["name"]},{similarity}\n')

In [1]:
# Calculate the similarity between each pair of images with ssim
with open('ssim_similarity.txt', 'w') as f:
    for i in range(len(pokemon_list)):
        for j in range(i + 1, len(pokemon_list)):
            pokemon1 = pokemon_list[i]
            pokemon2 = pokemon_list[j]
            image1 = cv2.imread(pokemon1['image_path'], cv2.IMREAD_GRAYSCALE)
            image2 = cv2.imread(pokemon2['image_path'], cv2.IMREAD_GRAYSCALE)

            # Resize images to the same dimensions
            image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

            # Calculate the SSIM value between the images
            ssim = cv2.compare_ssim(image1, image2)

            f.write(f'{pokemon1["name"]} - {pokemon2["name"]}: {ssim}\n')

NameError: name 'pokemon_list' is not defined

In [ ]:
import json
import cv2

# Load the JSON files containing the Pokemon data
with open('pokedex_with_images.json', 'r') as f:
    pokemon_list = json.load(f)

# Calculate the similarity between each pair of images
similarity_scores = {}
for i in range(len(pokemon_list)):
    for j in range(i + 1, len(pokemon_list)):
        pokemon1 = pokemon_list[i]
        pokemon2 = pokemon_list[j]
        image1 = cv2.imread(pokemon1['image_path'])
        image2 = cv2.imread(pokemon2['image_path'])

        # Resize images to the same dimensions
        image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

        # Calculate the structural similarity index
        ssim_score = cv2.compare_ssim(image1, image2)

        # Add the similarity score to the dictionary
        pokemon1_name = pokemon1['name']
        pokemon2_name = pokemon2['name']
        similarity_scores[(pokemon1_name, pokemon2_name)] = ssim_score

# Define a function to find the most similar Pokemon to a given Pokemon
def find_most_similar_pokemon(pokemon_name):
    most_similar_pokemon = None
    highest_similarity_score = -1
    for other_pokemon in pokemon_list:
        if other_pokemon['name'] == pokemon_name:
            continue
        other_pokemon_name = other_pokemon['name']
        similarity_score = similarity_scores.get((pokemon_name, other_pokemon_name))
        if similarity_score is not None and similarity_score > highest_similarity_score:
            highest_similarity_score = similarity_score
            most_similar_pokemon = other_pokemon
    return most_similar_pokemon, highest_similarity_score

# Define a function to find the ancestors of a given Pokemon
def find_ancestors(pokemon_name):
    ancestors = []
    current_pokemon_name = pokemon_name
    while True:
        current_pokemon, similarity_score = find_most_similar_pokemon(current_pokemon_name)
        if current_pokemon is None or similarity_score < 0.9:
            break
        ancestors.append(current_pokemon)
        current_pokemon_name = current_pokemon['name']
    return ancestors

# Find the ancestors of Pikachu
pikachu_ancestors = find_ancestors('Pikachu')
print('Ancestors of Pikachu:')
for ancestor in pikachu_ancestors:
    print('-', ancestor['name'])
